In [1]:
from dotenv import load_dotenv
import os
import time
from datetime import datetime
import math

import cv2
import csv
import numpy as np
import mediapipe as mp
import json

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import subprocess

from flask import Flask, Response, render_template_string

In [2]:
load_dotenv()
login, password = os.getenv("LOGIN"), os.getenv("PASSWORD")
login

'65973'

In [3]:
def finger_is_open(wrist, tip, pip, coeff):
    """
    Считаем палец "открытым", расстояние от кисти до кончика пальца больше, 
    чем (расстояние от кисти до первой фаланги пальца) * 1.45.
    """


    def powDist(a, b):
        return ((a.x - b.x)*(a.x - b.x) + (a.y - b.y)*(a.y - b.y))
    #return [int(powDist(wrist, pip) * coeff*coeff < powDist(wrist, tip)), str(powDist(wrist, pip) * coeff*coeff)[:4], str(powDist(wrist, tip))[:4]]
    #print(powDist(wrist, pip) * coeff*coeff, powDist(wrist, tip))
    return int(powDist(wrist, pip) * coeff*coeff < powDist(wrist, tip))


def open_fingers(hand_landmarks):
    """
    Возвращает количество "открытых" пальцев для одной руки.
    """
    finger_tips = [4, 8, 12, 16, 20]
    finger_pips = [2, 5, 9, 13, 17]
    coeffForFingers = [1.5, 1.6, 1.8, 1.8, 1.6]

    open_finger = []
    
    for tip_id, pip_id, coeff_id in zip(finger_tips, finger_pips, coeffForFingers):
        open_finger.append(finger_is_open(hand_landmarks.landmark[0], hand_landmarks.landmark[tip_id], hand_landmarks.landmark[pip_id], coeff_id))

    return open_finger


def classify_gesture(hand_landmarks):
    """
    Классификация жеста по количеству открытых пальцев и их относительному расположению.
    """
    list_open_fingers = open_fingers(hand_landmarks)
    #return f"{list_open_fingers}"

    count_open_fingers = sum(list_open_fingers)
    
    if count_open_fingers == 0:
        return f"Fist"
    elif count_open_fingers == 5:
        return f"Open Hand"
    elif count_open_fingers == 1 and list_open_fingers[1] == 1:
        return f"Ukazannie"
    elif count_open_fingers == 2 and list_open_fingers[1] == 1 and list_open_fingers[2] == 1: 
        return f"Victory"
    elif count_open_fingers == 3 and list_open_fingers[0] == 1 and list_open_fingers[1] == 1 and list_open_fingers[2] == 1: 
        return f"TiDishi"
    elif count_open_fingers == 2 and list_open_fingers[0] == 1 and list_open_fingers[4] == 1: 
        return f"Jambo"
    elif count_open_fingers == 3 and list_open_fingers[0] == 1 and list_open_fingers[1] == 1 and list_open_fingers[4] == 1: 
        return f"Rock"
    else:
        return f"{list_open_fingers}: {count_open_fingers}"

In [7]:
ADB_PATH = "C:\Program Files\BlueStacks_nxt\HD-Adb.exe"
BLUESTACKS_HOST = "127.0.0.1:5555"   # он же device-id

def adb(*args, use_device=True):
    """
    Обёртка над adb.
    use_device=False — для команд типа connect/devices, где -s не нужен.
    """
    cmd = [ADB_PATH]
    if use_device:
        cmd += ["-s", BLUESTACKS_HOST]
    cmd += list(args)
    print("CMD:", " ".join(cmd))
    return subprocess.check_output(cmd, encoding="utf-8")

def tap(x, y):
    adb("shell", "input", "tap", str(x), str(y))

In [ ]:
def get_stream_url_via_selenium():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--window-size=1920,1080")

    LOGIN_URL = "https://video.2090000.ru/login.html"
    CAMERA_URL = "https://video.2090000.ru/account/camera/3104703/view.html?backPage=1"
    STREAM_API_URL = "https://video.2090000.ru/account/camera/3104703/url.html?speed_mul=1&time=&timeZoneOffset=25200&format=hls&isSingleCamera=1&="

    driver = webdriver.Edge(options=options)
    wait = WebDriverWait(driver, 15)

    try:
        # логин
        
        driver.get(LOGIN_URL)
        wait.until(EC.presence_of_element_located((By.ID, "username")))

        driver.find_element(By.NAME, "User[Login]").send_keys(login)
        driver.find_element(By.ID, "password").send_keys(password, Keys.ENTER)

        wait.until(EC.url_contains("/account/view.html"))

        # идём по прямой ссылке на API, которое возвращает JSON с URL потока
        driver.get(CAMERA_URL)
        wait.until(
            EC.any_of(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, ".error-text")),
                EC.invisibility_of_element_located((By.CSS_SELECTOR, ".spinner"))
            )
        )
        WebDriverWait(driver, 15).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, ".img-div video"))
        )
        print(STREAM_API_URL)
        driver.get(STREAM_API_URL+str(time.time()))
        # страница, скорее всего, просто отдаёт текст JSON
        # можно взять так:
        body_text = driver.find_element(By.TAG_NAME, "body").text
        #print("DEBUG body_text:", repr(body_text[:300]))
        data = json.loads(body_text)
        if not data.get("Status") or data.get("Error"):
            raise RuntimeError(f"API вернул ошибку: {data}")

        stream_url = data["URL"]
        print("Получен URL потока:", stream_url)
        return stream_url
    finally:
        driver.quit()

In [18]:
# ==== НАСТРОЙКИ ====

FRAME_INTERVAL = 1/20
global current_fps

# ==== MediaPipe и жесты ====

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


# Инициализируем MediaPipe Hands (для видео лучше static_image_mode=False)

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5
)


# ==== Flask-приложение ====

app = Flask(__name__)


@app.route("/")
def index():
    return render_template_string("""
    <!doctype html>
    <html>
    <head>
        <title>Живой видеопоток с жестами</title>
        <meta charset="utf-8">
        <style>
            body {
                background: #111;
                color: #eee;
                font-family: sans-serif;
            }
            h1 {
                text-align: center;
                margin-top: 15px;
            }
            .wrapper {
                display: flex;
                flex-direction: row;
                align-items: flex-start;
                justify-content: center;
                margin-top: 20px;
                gap: 20px;
            }
            .sidebar {
                min-width: 220px;
                padding: 10px 15px;
                border: 1px solid #444;
                border-radius: 8px;
                background: #181818;
            }
            .sidebar h2 {
                margin-top: 0;
                font-size: 18px;
                margin-bottom: 10px;
            }
            .metric {
                font-size: 15px;
                margin-bottom: 8px;
            }
            #gesture-list {
                list-style: none;
                padding-left: 0;
                margin: 4px 0 0 0;
                font-size: 14px;
            }
            #gesture-list li {
                margin-bottom: 2px;
            }
            img {
                max-width: 70vw;
                border: 2px solid #555;
            }
            .status-circle {
                width: 18px;
                height: 18px;
                border-radius: 50%;
                margin-bottom: 6px;
                background: #700; /* по умолчанию красный */
                box-shadow: 0 0 6px rgba(0, 0, 0, 0.7);
                border: 1px solid #333;
            }
            .status-circle.ok {
                background: #0a0; /* зелёный */
            }
            .status-circle.bad {
                background: #700; /* красный */
            }
        </style>
    </head>
    <body>
        <h1>Отладочный стрим</h1>

        <div class="wrapper">
            <div class="sidebar">
                <div id="door-indicator" class="status-circle bad"></div>
                <h2>Статистика</h2>
                <div class="metric">
                    FPS: <span id="fps-value">--</span>
                </div>
                <div class="metric">
                    Всего жестов: <span id="gesture-total">0</span>
                </div>
                <div class="metric">
                    Жесты:
                    <ul id="gesture-list"></ul>
                </div>
            </div>

            <div class="video">
                <img src="{{ url_for('video_feed') }}" />
            </div>
        </div>

        <script>
            const fpsSpan     = document.getElementById("fps-value");
            const openTimeSpan= document.getElementById("open-time-value");
            const doorIndicator = document.getElementById("door-indicator");
            const totalSpan   = document.getElementById("gesture-total");
            const gestureList = document.getElementById("gesture-list");

            const statsSource = new EventSource("/stats_stream");

            function renderGestureList(counts) {
                gestureList.innerHTML = "";
                Object.entries(counts).forEach(([name, count]) => {
                    const li = document.createElement("li");
                    li.textContent = name + ": " + count;
                    gestureList.appendChild(li);
                });
            }

            statsSource.onmessage = (event) => {
                try {
                    const data = JSON.parse(event.data); // {fps: ..., gesture: [...]}

                    // FPS -> int
                    if ("fps" in data && typeof data.fps === "number") {
                        fpsSpan.textContent = Math.round(data.fps);
                    }

                    // gesture -> массив строк
                    let gestures = [];
                    if (Array.isArray(data.gesture)) {
                        gestures = data.gesture;
                    }
                                  
                    if ("open_time" in data && typeof data.open_time === "number") {
                        //openTimeSpan.textContent = Math.round(data.open_time);
                        const nowSec = Date.now() / 1000;
                        const diff = nowSec - data.open_time;
                        //console.log(openTimeSpan);
                        // < 7 секунд -> зелёный, иначе красный
                        if (diff < 9) {
                            doorIndicator.classList.add("ok");
                            doorIndicator.classList.remove("bad");
                            //console.log("green");
                        } else {
                            doorIndicator.classList.add("bad");
                            doorIndicator.classList.remove("ok");
                            //console.log("red");
                        }
                    }
                    

                    // Непустые значения
                    const nonEmpty = gestures
                        .map(g => String(g).trim())
                        .filter(g => g !== "");

                    // Счётчик по типам жестов
                    const gestureCounts = {};
                    for (const g of nonEmpty) {
                        gestureCounts[g] = (gestureCounts[g] || 0) + 1;
                    }

                    // Сумма всех вхождений
                    const totalGestures = nonEmpty.length;
                    totalSpan.textContent = totalGestures;

                    renderGestureList(gestureCounts);
                } catch (e) {
                    console.error("Ошибка парсинга stats_stream:", e, event.data);
                }
            };

            statsSource.onerror = (err) => {
                console.error("SSE error", err);
            };
        </script>
    </body>
    </html>
    """)



current_stats = {}
def gen_frames_novotelecom(source_vebka = False):

    SCREENSHOTS_ROOT = "screenshots_test"   # корневая папка для сохранения
    RESULTS_CSV = "gesture_results.csv"
   
    # ==== MediaPipe и жесты ====

    findGesture = []
    first_time = time.time()
    times = []
    global current_fps
    last_open = 0


    # ==== Selenium: запускаем браузер ====

    # Открываем CSV заранее
    csvfile = open(RESULTS_CSV, mode="w", newline="", encoding="utf-8")
    writer = csv.writer(csvfile, delimiter=";")
    writer.writerow(["timestamp", "filename", "hand_index", "handedness", "gesture"])


    try:
        if not(source_vebka):
            stream_URL = get_stream_url_via_selenium()
            cap = cv2.VideoCapture(stream_URL)
        else:
            cap = cv2.VideoCapture(0)
        
        adb("connect", BLUESTACKS_HOST, use_device=False)
        print("Начинаю захват и анализ кадров... (Ctrl+C чтобы остановить)")

        while True:
            now = datetime.now()
            # Для имени файла и логов
            timestamp_str = now.strftime("%Y-%m-%d_%H-%M-%S.%f")[:-3]  # до миллисекунд

            # Делаем скриншот элемента <video> в PNG (байты)
            ret, frame_bgr = cap.read()
            if not ret or frame_bgr is None:
                print("Не удалось прочитать кадр с веб-камеры, пропускаю кадр")
                break

            # BGR -> RGB для MediaPipe
            frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

            # Анализ руками
            results = hands.process(frame_rgb)

            # Будем рисовать на копии исходного кадра
            output_frame = frame_bgr.copy()

            times.append(round(time.time() - first_time, 2))
            if len(times) > 20:
                times.pop(0)
                current_stats['fps'] = (20/(times[-1] - times[0]))
            
            gesture_name = ""

            if results.multi_hand_landmarks and results.multi_handedness:
                # Найдены руки
                # gestures_for_filename = []

                for idx, (hand_landmarks, hand_handedness) in enumerate(
                    zip(results.multi_hand_landmarks, results.multi_handedness)
                ):
                    #label = hand_handedness.classification[0].label  # "Left" / "Right"
                    label = 0
                    # gesture_name = classify_gesture(hand_landmarks, label)
                    gesture_name = classify_gesture(hand_landmarks)
                    
                    # gestures_for_filename.append(f"{label}_{gesture_name}")

                    # Рисуем скелет руки
                    mp_drawing.draw_landmarks(
                        output_frame,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS
                    )

                    # Подпишем жест около запястья
                    h, w, _ = output_frame.shape
                    wrist = hand_landmarks.landmark[0]
                    x = int(wrist.x * w)
                    y = int(wrist.y * h)

                    cv2.putText(
                        output_frame,
                        f"{label}: {gesture_name}",
                        (x, max(20, y - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.7,
                        (0, 255, 0),
                        2
                    )

                    # Запись в CSV
                    writer.writerow([
                        timestamp_str,
                        "",            # имя файла подставим позже, если сохраним
                        idx,
                        label,
                        gesture_name
                    ])

                # Если хотя бы одна рука найдена — сохраняем кадр

            findGesture.append(gesture_name)
            current_stats['gesture'] = (findGesture)
            if len(findGesture) > 20:
                findGesture.pop(0)
                if findGesture.count("TiDishi") >= 5:
                    #gesture_str = "-".join(gestures_for_filename)
                    
                    if time.time() - last_open > 7: # Интервал между нажатиями должен быть больше 7 секунд
                        print(f"откртыие двери в {now.strftime('%H-%M-%S')}")
                        tap(300, 616) # Открытие двери с помощью тапа по экрану
                        last_open = time.time()
                        current_stats["open_time"] = last_open
                    # Структура папок: screenshots/HH/MM/
                    hour_dir = now.strftime("%H")
                    output_dir = os.path.join(SCREENSHOTS_ROOT, hour_dir)
                    os.makedirs(output_dir, exist_ok=True)

                    filename = f"{now.strftime('%H-%M-%S')}.png"
                    output_path = os.path.join(output_dir, filename)

                    cv2.imwrite(output_path, output_frame)
                    # print(f"[{timestamp_str}] Найдены жесты: {gesture_str} -> {output_path}")
                        

                # Дописываем имя файла в последнюю строку CSV (если нужно прямо связать)
                # Проще: можно сразу писать filename в writer.writerow выше,
                # тут для простоты оставим как есть.

            # Ждём до следующего кадра
            ret, buffer = cv2.imencode('.jpg', output_frame)
            if not ret:
                continue
            frame = buffer.tobytes()

            # MJPEG-ответ: каждый кадр — отдельная часть multipart-потока
            
            yield (b'--frame\r\n'
                    b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            #time.sleep(FRAME_INTERVAL)

    except KeyboardInterrupt:
        print("Остановка по Ctrl+C")

    finally:
        #hands.close()
        csvfile.close()
        cap.release()
        cv2.destroyAllWindows()
        adb("disconnect", BLUESTACKS_HOST, use_device=False)
        print("Завершено.")


def gen_frames():
    """
    Генератор JPEG-кадров для браузера.
    Тут же делаем анализ рук и рисуем разметку.
    """
    # ==== Камера ====
    #stream_URL = get_stream_url_via_selenium()
    #cap = cv2.VideoCapture(stream_URL)

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("Не удалось открыть веб-камеру")
    
    while True:
        now = datetime.now()
        timestamp_str = now.strftime("%Y-%m-%d_%H-%M-%S.%f")[:-3]

        ret, frame_bgr = cap.read()
        if not ret or frame_bgr is None:
            print("Не удалось прочитать кадр с веб-камеры, пропускаю кадр")
            break

        # BGR -> RGB для MediaPipe
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

        # Анализ руками
        results = hands.process(frame_rgb)

        # Будем рисовать на копии исходного кадра
        output_frame = frame_bgr.copy()

        if results.multi_hand_landmarks and results.multi_handedness:
            gestures_for_filename = []

            for idx, (hand_landmarks, hand_handedness) in enumerate(
                zip(results.multi_hand_landmarks, results.multi_handedness)
            ):
                label = hand_handedness.classification[0].label  # "Left" / "Right"
                gesture_name = classify_gesture(hand_landmarks)

                gestures_for_filename.append(f"{label}_{gesture_name}")

                # Рисуем скелет руки
                mp_drawing.draw_landmarks(
                    output_frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS
                )

                # Подпишем жест около запястья
                h, w, _ = output_frame.shape
                wrist = hand_landmarks.landmark[0]
                x = int(wrist.x * w)
                y = int(wrist.y * h)

                cv2.putText(
                    output_frame,
                    f"{label}: {gesture_name}",
                    (x, max(20, y - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.4,
                    (0, 255, 0),
                    2
                )

        # Кодируем кадр в JPEG для отправки в браузер
        ret, buffer = cv2.imencode('.jpg', output_frame)
        if not ret:
            continue
        frame = buffer.tobytes()

        # MJPEG-ответ: каждый кадр — отдельная часть multipart-потока
        
        yield (b'--frame\r\n'
                b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
        

        # Ограничиваем частоту кадров
        #time.sleep(FRAME_INTERVAL)


@app.route("/video_feed")
def video_feed():
    print("Запрошен видеопоток")
    return Response(
        gen_frames_novotelecom(source_vebka=True),
        mimetype='multipart/x-mixed-replace; boundary=frame'
    )

@app.route("/stats_stream")
def stats_stream():
    def event_stream():
        while True:
            yield f"data: {json.dumps(current_stats, ensure_ascii=False)}\n\n"
            time.sleep(0.1)  # как часто обновлять FPS на странице

    return Response(event_stream(), mimetype="text/event-stream")


if __name__ == "__main__":
    try:
        # host="0.0.0.0" — чтобы открыть со всех устройств в локалке
        app.run(host="0.0.0.0", port=5000, debug=False)
    finally:
        # Корректно закрываем ресурсы при завершении
        cv2.destroyAllWindows()
        print("Flask-сервер остановлен.")
        hands.close()
        #cap.release()
                



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.18.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Dec/2025 21:43:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2025 21:43:09] "GET /stats_stream HTTP/1.1" 200 -


Запрошен видеопоток
CMD: C:\Program Files\BlueStacks_nxt\HD-Adb.exe connect 127.0.0.1:5555
Начинаю захват и анализ кадров... (Ctrl+C чтобы остановить)


127.0.0.1 - - [05/Dec/2025 21:43:10] "GET /video_feed HTTP/1.1" 200 -


откртыие двери в 21-43-59
CMD: C:\Program Files\BlueStacks_nxt\HD-Adb.exe -s 127.0.0.1:5555 shell input tap 300 616
откртыие двери в 21-50-31
CMD: C:\Program Files\BlueStacks_nxt\HD-Adb.exe -s 127.0.0.1:5555 shell input tap 300 616
откртыие двери в 21-51-05
CMD: C:\Program Files\BlueStacks_nxt\HD-Adb.exe -s 127.0.0.1:5555 shell input tap 300 616
откртыие двери в 21-51-49
CMD: C:\Program Files\BlueStacks_nxt\HD-Adb.exe -s 127.0.0.1:5555 shell input tap 300 616
Flask-сервер остановлен.


Error on request:
Traceback (most recent call last):
  File "c:\Users\Michail\AppData\Local\Programs\Python\Python310\lib\site-packages\werkzeug\serving.py", line 370, in run_wsgi
    execute(self.server.app)
  File "c:\Users\Michail\AppData\Local\Programs\Python\Python310\lib\site-packages\werkzeug\serving.py", line 333, in execute
    for data in application_iter:
  File "c:\Users\Michail\AppData\Local\Programs\Python\Python310\lib\site-packages\werkzeug\wsgi.py", line 256, in __next__
    return self._next()
  File "c:\Users\Michail\AppData\Local\Programs\Python\Python310\lib\site-packages\werkzeug\wrappers\response.py", line 32, in _iter_encoded
    for item in iterable:
  File "C:\Users\Michail\AppData\Local\Temp\ipykernel_18372\2564184135.py", line 253, in gen_frames_novotelecom
    results = hands.process(frame_rgb)
  File "c:\Users\Michail\AppData\Local\Programs\Python\Python310\lib\site-packages\mediapipe\python\solutions\hands.py", line 153, in process
    return super().proc

CMD: C:\Program Files\BlueStacks_nxt\HD-Adb.exe disconnect 127.0.0.1:5555
Завершено.
